In [1]:
from django.shortcuts import render
import os
from PyPDF2 import PdfFileMerger
# authendication
from rest_framework.decorators import api_view, permission_classes
from django.contrib.auth import authenticate, logout, login
from rest_framework.authtoken.models import Token
from rest_framework.permissions import AllowAny
from django.contrib.auth.hashers import make_password
# Create your views here.
from django.db.models import Max
from collections import defaultdict, OrderedDict, Counter
# creating views for REST / function based views
from rest_framework import status
from rest_framework.response import Response
from django.contrib.auth import authenticate, logout, login
from django.contrib.auth.models import User
import random
import plivo
from main.models import *
import pandas as pd
import numpy as np
from decimal import Decimal
from datetime import timedelta, date
# from datetime import datetime
import datetime
import dateutil.relativedelta
from base64 import b64encode, b64decode
#  Plivo credentials
# import plivo
from random import randint
from django.core.files.base import ContentFile
from django.db.models import Sum, Max
from pytz import timezone
from django.db.models import Sum
from calendar import monthrange
from Google import Create_Service
from calendar import monthrange, month_name

# canvas
from reportlab.pdfgen import canvas
from reportlab.platypus import BaseDocTemplate, SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib.pagesizes import letter
from reportlab.lib.units import inch

from reportlab.lib import colors
from main.pdf_generate import *
from main.temp_pdf_generate import generate_pdf_for_temp_route
import csv

In [2]:
def daily_cash_collection_report_counter_wise_zone():
    date = "2020-09-3"
    data_dict = {}
    grand_total_dict = {}
    
    counter_obj = Counter.objects.filter(is_active=True)

    for zone in Zone.objects.all().exclude(id__in=[11,12,13,15]):
        if zone.name not in grand_total_dict:
            grand_total_dict[zone.name] = 0
    grand_total_dict["Total"] = 0
            
    for counter in counter_obj:
        data_dict[counter.name] = {}
        for zone in Zone.objects.all().exclude(id__in=[11,12,13,15]):
            if zone.name not in data_dict[counter.name]:
                data_dict[counter.name][zone.name] = 0
        data_dict[counter.name]["Total"] = 0
                
        agent_emp_trace = list(CounterEmployeeTraceMap.objects.filter(counter_id=counter.id,time_created__date=date).values_list("id",flat=True))
        icustomer_emp_trace = list(CounterEmployeeTraceMap.objects.filter(counter_id=counter.id,collection_date=date).values_list("id",flat=True))
        
        agent_trace_sale_group_list = list(CounterEmployeeTraceSaleGroupMap.objects.filter(counter_employee_trace_id__in=agent_emp_trace).values_list('sale_group', flat=True))
        customer_trace_sale_group_list = list(CounterEmployeeTraceSaleGroupMap.objects.filter(counter_employee_trace_id__in=icustomer_emp_trace).values_list('icustomer_sale_group', flat=True))
        
        icustomer_sale_group_obj = ICustomerSaleGroup.objects.filter(id__in=customer_trace_sale_group_list).order_by('id')
        sale_group_obj = SaleGroup.objects.filter(id__in=agent_trace_sale_group_list).order_by('id')
        
        for agent_sale in sale_group_obj:
            if agent_sale.zone.name in data_dict[counter.name]:
                data_dict[counter.name][agent_sale.zone.name] += agent_sale.total_cost
                data_dict[counter.name]["Total"] += agent_sale.total_cost
                
        for customer_sale in icustomer_sale_group_obj:
            if customer_sale.zone.name in data_dict[counter.name]:
                data_dict[counter.name][customer_sale.zone.name] += customer_sale.total_cost_for_month
                data_dict[counter.name]["Total"] += customer_sale.total_cost_for_month
                
    data_dict["Online Agent"] = {}
    data_dict["Online Customer"] = {}
    
    for counter in ["Online Agent","Online Customer"]:
        for zone in Zone.objects.all().exclude(id__in=[11,12,13,15]):
            if zone.name not in data_dict[counter]:
                data_dict[counter][zone.name] = 0
        data_dict[counter]["Total"] = 0
                
    for zone in Zone.objects.all().exclude(id__in=[11,12,13,15]):
        agent_online_obj = SaleGroup.objects.filter(time_created__date=date, ordered_via_id__in=[1, 3],zone_id=zone.id).order_by('id')
        card_online_obj = ICustomerSaleGroup.objects.filter(time_created__date=date, ordered_via_id__in=[1, 3],zone_id=zone.id).order_by('id')
        
        for agent_sale in agent_online_obj:
            data_dict["Online Agent"][zone.name] += agent_sale.total_cost
            data_dict["Online Agent"]["Total"] += agent_sale.total_cost
            
        for customer_sale in card_online_obj:
            data_dict["Online Customer"][zone.name] += customer_sale.total_cost_for_month
            data_dict["Online Customer"]["Total"] += customer_sale.total_cost_for_month
            
    for data in data_dict:
        for datas in data_dict[data]:
            grand_total_dict[datas] += data_dict[data][datas]
            

    data = pdf_gen_for_daily_cash_collection_report_counter_wise_zone(data_dict,grand_total_dict,date)
    return data

In [3]:
def pdf_gen_for_daily_cash_collection_report_counter_wise_zone(data_dict,grand_total_dict,date):
    date = date
    date_in_format = datetime.datetime.strptime(date, '%Y-%m-%d')
    date = datetime.datetime.strftime(date_in_format, '%d-%m-%Y')
#     datetime.datetime.strftime(date, '%d-%b-%Y')
    file_name = "daily_cash_collection_report_counter_wise_zone_for_"+str(date) + '.pdf'
    file_path = os.path.join('static/media/', file_name)
    mycanvas = canvas.Canvas(file_path, pagesize=(15 * inch, 12 * inch))
    
    mycanvas.setFont('Helvetica', 15)
    mycanvas.drawCentredString(560, 820, 'The Coimbatore District Co-Operative Milk Producers Union Ltd, Pachapalayam, Coimbatore - 641 010')
    mycanvas.setFont('Helvetica', 15)
   
    mycanvas.drawCentredString(560, 805, 'Daily Cash Collection Counter Wise Zone Sales For - '+str(date))
#     mycanvas.line(226-50, 802, 473-50, 802)
    
    df = pd.DataFrame(data_dict)
    df_t = df.T
    df_t.to_excel("sunesh.xlsx")
    df_ex = pd.read_excel("sunesh.xlsx",header=None)
    df_ex =df_ex.fillna('Counter')
    
    new_header = df_ex.iloc[0] 
    df_ex = df_ex[1:]
    df_ex.columns = new_header
    
    data_dict = df_ex.to_dict('r')
    
    x_total_len = 1060
    x_axis = 0
    x_axis_line = 10
    y_axis = 690+80
    y_axis_line = 690+60
    len_adjust=len(data_dict[0])
    
    mycanvas.line(x_axis_line,y_axis+20,x_total_len+10,y_axis+20)
    mycanvas.line(x_axis_line,y_axis-20,x_total_len+10,y_axis-20)
    
    for data in data_dict[0]:
        mycanvas.drawCentredString(x_axis+60,y_axis-5,str(data))
        x_axis += (x_total_len/len_adjust)
    x_axis = 0
    for data in data_dict:
        if data["Total"] != 0:
            for stock_name in data_dict[0]:
                if stock_name == "Counter":
                    mycanvas.setFont('Helvetica', 9)
                    mycanvas.drawString(x_axis+11,y_axis-40,str(data[stock_name]))
                else:
                    mycanvas.setFont('Helvetica', 15)
                    mycanvas.drawRightString(x_axis+125,y_axis-40,str(data[stock_name]))
                mycanvas.line(x_axis+10,y_axis+20,x_axis+10,y_axis-90)
                mycanvas.line(x_total_len+10,y_axis+20,x_total_len+10,y_axis-90)
                x_axis += (x_total_len/len_adjust)
            x_axis = 0
            y_axis -= 25
    mycanvas.line(x_axis_line,y_axis-30,x_total_len+10,y_axis-30)
    mycanvas.line(x_axis_line,y_axis-65,x_total_len+10,y_axis-65)
    
    mycanvas.drawCentredString(x_axis+65,y_axis-55,"GRAN TOTAL")
    
    x_axis = 0
    for data in grand_total_dict:
        mycanvas.drawRightString(x_axis+125+(x_total_len/len_adjust),y_axis-55,str(grand_total_dict[data]))
        x_axis += (x_total_len/len_adjust)
        
    print(grand_total_dict)
    mycanvas.save()

In [4]:
daily_cash_collection_report_counter_wise_zone()

{'EAST': Decimal('1579623.10'), 'WEST': Decimal('1602469.50'), 'SOUTH': Decimal('1518152.60'), 'POLLACHI': Decimal('130567.00'), 'NORTH': Decimal('1758001.20'), 'MTP': Decimal('480724.00'), 'CURD Zone': Decimal('2800.00'), 'Total': Decimal('7072337.40')}


/home/suneshrajan/Software/Virtualenv/aavin/lib/python3.8/site-packages/pandas/core/frame.py:1485: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  warnings.warn(
